In [5]:
import pandas as pd
import os
import shutil

In [48]:
basedir = '/data/CARD/PD/GP2/raw_genotypes'
out_genotypes = '/data/CARD/PD/GP2/genotypes'
shulman_ny_path = f'{basedir}/shulman_ny'
gtc_file_path = f'{shulman_ny_path}/GP2_GCT_files'
idat_file_path = f'{shulman_ny_path}/GP2_Shulman'
key_file = f'{gtc_file_path}/Key File_FINAL_Shulman_and_NY_011421.txt'
manifest_txt_path = f'{gtc_file_path}/FINALSS_after_rerun__Shulman_and_NY_011421.csv'
bpm = f'{gtc_file_path}/NeuroBooster_20042459_A1.bpm'
cluster_file = f'{gtc_file_path}/NBSCluster_file_n1393_011921.egt'

#software paths
GTCtoVCF = 'GTCtoVCF/gtc_to_vcf.py'
iaap = 'iaap-cli-linux-x64-1.1.0-sha.80d7e5b3d9c1fdfc2e99b472a90652fd3848bbc7/iaap-cli/iaap-cli'

ref_fasta = 'ref_data/hg38_ref.fa'
shulman_gtc_path = f'{basedir}/SHULMAN/gtc_files'
shulman_idat_path = f'{basedir}/SHULMAN/idats'
shulman_out = f'{out_genotypes}/SHULMAN'
ny_gtc_path = f'{basedir}/NY/gtc_files'
ny_idat_path = f'{basedir}/NY/idats'
ny_out = f'{out_genotypes}/NY'


In [7]:
manifest = pd.read_csv(manifest_txt_path, header=10)
# key = pd.read_csv(key_file, sep='\t')
manifest

,Sample_ID,Plate Number,Sample_Well,SentrixBarcode_A,SentrixPosition_A,Gender,Study,Batch Id
0,6723,1,A1,204697840024,R01C01,Male,Shulman,1
1,7522,1,B1,204697840024,R02C01,Male,Shulman,1
2,7525,1,C1,204697840024,R03C01,Male,Shulman,1
3,7532,1,D1,204697840024,R04C01,Female,Shulman,1
4,7539,1,E1,204697840024,R05C01,Male,Shulman,1
...,...,...,...,...,...,...,...,...
1388,MBA_LSDA_101,2,E1,204701870002,R05C01,Female,NY,8
1389,MBA_LSDA_102,2,F1,204701870002,R06C01,Female,NY,8
1390,MBA_LSDA_103,2,G1,204701870002,R07C01,Male,NY,8
1391,MBA_LSDA_104,2,H1,204701870002,R08C01,Male,NY,8


In [25]:
# create new directories to store split cohorts
!mkdir {basedir}/SHULMAN
!mkdir {basedir}/NY
!mkdir {basedir}/SHULMAN/gtc_files
!mkdir {basedir}/NY/gtc_files
!mkdir {basedir}/SHULMAN/idats
!mkdir {basedir}/NY/idats
!mkdir {out_genotypes}
!mkdir {out_genotypes}/SHULMAN
!mkdir {out_genotypes}/SHULMAN/vcfs
!mkdir {out_genotypes}/SHULMAN/peds
!mkdir {out_genotypes}/NY
!mkdir {out_genotypes}/NY/vcfs
!mkdir {out_genotypes}/NY/peds


mkdir: cannot create directory '/data/CARD/PD/GP2/raw_genotypes/SHULMAN': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/raw_genotypes/NY': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/raw_genotypes/SHULMAN/gtc_files': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/raw_genotypes/NY/gtc_files': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/raw_genotypes/NY/idats': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/genotypes': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/genotypes/SHULMAN': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/genotypes/SHULMAN/vcfs': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/genotypes/NY': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/genotypes/NY/vcfs': File exists


In [8]:
# create filenaames and split manifest into respective cohorts
manifest['filename'] = manifest['SentrixBarcode_A'].astype(str) + '_' + manifest['SentrixPosition_A']
shulman = manifest.loc[manifest.Study == 'Shulman']
ny = manifest.loc[manifest.Study == 'NY']

In [38]:
# copy files to respective directories
# for filename in shulman.filename:
#     shutil.copyfile(src=f'{gtc_file_path}/{filename}.gtc',dst=f'{basedir}/SHULMAN/gtc_files/{filename}.gtc')
# for filename in ny.filename:
#     shutil.copyfile(src=f'{gtc_file_path}/{filename}.gtc',dst=f'{basedir}/NY/gtc_files/{filename}.gtc')   

In [39]:
# now clone ILMN GTCtoVCF github repo
# !git clone https://github.com/Illumina/GTCtoVCF.git
# get hg38 reference
# !wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.26_GRCh38/GCF_000001405.26_GRCh38_genomic.fna.gz -P ref_data
# !gunzip ref_data/GCF_000001405.26_GRCh38_genomic.fna.gz

In [9]:
# swarm command to run in parallel
with open('shulman_gtc_to_vcf.swarm','w') as f:
    
    for filename in shulman.filename:
        
        shulman_gtc_to_vcf_cmd = f'\
python3 {GTCtoVCF} \
--gtc-paths {shulman_gtc_path}/{filename}.gtc \
--manifest-file {bpm} \
--genome-fasta-file {ref_fasta} \
--output-vcf-path {shulman_out}/vcfs/{filename}.vcf \
--skip-indels'
        
        f.write(f'{shulman_gtc_to_vcf_cmd}\n')
f.close()


In [10]:
# !swarm -f shulman_gtc_to_vcf.swarm -g 16 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

In [23]:
# swarm command to run in parallel
with open('ny_gtc_to_vcf.swarm','w') as f:
    
    for filename in ny.filename:
        
        ny_gtc_to_vcf_cmd = f'\
python3 {GTCtoVCF} \
--gtc-paths {ny_gtc_path}/{filename}.gtc \
--manifest-file {bpm} \
--genome-fasta-file {ref_fasta} \
--output-vcf-path {ny_out}/vcfs/{filename}.vcf \
--skip-indels'
        
        f.write(f'{ny_gtc_to_vcf_cmd}\n')
f.close()


In [24]:
!swarm -f ny_gtc_to_vcf.swarm -g 16 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

6960101


In [43]:
# call genotypes from idats so we can compare gtc to idats at the end
# create directory for each sentrix barcode A if they don't already exist
for code in shulman.SentrixBarcode_A.unique():
    if os.path.exists(f'{basedir}/SHULMAN/idats/{code}'):
        print(f'{basedir}/SHULMAN/idats/{code} already exists')
    else:
        os.mkdir(f'{basedir}/SHULMAN/idats/{code}')

missing_idats = []

# split idats into respective cohorts and populated each sentrix barcode A directory
for i, filename in enumerate(shulman.filename):
    
    grn = f'{idat_file_path}/{shulman.SentrixBarcode_A.iloc[i]}/{filename}_Grn.idat'
    red = f'{idat_file_path}/{shulman.SentrixBarcode_A.iloc[i]}/{filename}_Red.idat'

    if os.path.isfile(grn):
        shutil.copyfile(src=grn, dst=f'{basedir}/SHULMAN/idats/{shulman.SentrixBarcode_A.iloc[i]}/{filename}_Grn.idat')
    else:
        missing_idats.append(grn)
        
    if os.path.isfile(red):
        shutil.copyfile(src=red, dst=f'{basedir}/SHULMAN/idats/{shulman.SentrixBarcode_A.iloc[i]}/{filename}_Red.idat')
    else:
        missing_idats.append(red)
    

len(missing_idats)

0

In [45]:
# create directory for each sentrix barcode A if they don't already exist
for code in ny.SentrixBarcode_A.unique():
    if os.path.exists(f'{basedir}/NY/idats/{code}'):
        print(f'{basedir}/NY/idats/{code} already exists')
    else:
        os.mkdir(f'{basedir}/NY/idats/{code}')
        
missing_idats = []
# spit idats into respective cohorts
for i, filename in enumerate(ny.filename):
    grn = f'{idat_file_path}/{ny.SentrixBarcode_A.iloc[i]}/{filename}_Grn.idat'
    red = f'{idat_file_path}/{ny.SentrixBarcode_A.iloc[i]}/{filename}_Red.idat'
#     print(f'{idat_file_path}/{shulman.SentrixBarcode_A.iloc[i]}/{filename}_Grn.idat')
#     print(f'{idat_file_path}/{shulman.SentrixBarcode_A.iloc[i]}/{filename}_Red.idat')
    if os.path.isfile(grn):
        shutil.copyfile(src=grn, dst=f'{basedir}/NY/idats/{ny.SentrixBarcode_A.iloc[i]}/{filename}_Grn.idat')
    else:
        missing_idats.append(grn)
        
    if os.path.isfile(red):
        shutil.copyfile(src=red, dst=f'{basedir}/NY/idats/{ny.SentrixBarcode_A.iloc[i]}/{filename}_Red.idat')
    else:
        missing_idats.append(red)
    

len(missing_idats)

0

In [60]:
# # swarm command to run in parallel
with open('shulman_idat_to_ped.swarm','w') as f:
    
    for code in shulman.SentrixBarcode_A.unique():
        
        shulman_idat_to_ped_cmd = f'\
{iaap} gencall \
{bpm} \
{cluster_file} \
{shulman_out}/peds/ \
-f {shulman_idat_path}/{code} \
-p \
-t 16'
        
        f.write(f'{shulman_gtc_to_vcf_cmd}\n')
f.close()


In [63]:
!swarm -f shulman_idat_to_ped.swarm -g 32 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

6984907
